### Classic:
In Riddler City, all the streets are currently two-way streets. But in an effort to make the metropolis friendlier for pedestrians and cyclists, the mayor has decreed that all streets should be one-way. Meanwhile, the civil engineer overseeing this transition is not particularly invested in the project and will be randomly assigning every block of each street a random direction.

For your daily commute to work, you drive a car two blocks east and two blocks south, as shown in the diagram below. What is the probability that, after each block is randomly assigned a one-way direction, there will still be a way for you to commute to work while staying within this two-by-two block region (i.e., sticking to the 12 streets you see in the diagram)? 


### General Approach: Graphs 

#### Graph Vertex:

```
0 1 2 

3 4 5

6 7 8

```

#### Combinations of Arcs:

Helpful to store as a dictionary. The graph traversal overview here was very helpful: https://www.python.org/doc/essays/graphs/

#### Randomization:

Prior to storing the graph as a dictionary, randomly change each edge (using 0/1 to do this)

#### Solution:

- Convert to a graph
- Edges become arcs (we have direction)
- Randomly switch direction of arc
- Confirm traversal from upper-left vertex to bottom-right vertex 


In [1]:
from numpy.random import binomial as bi
from collections import defaultdict

l_tuples = [(0,1), (0,3),
            (1,2), (1,4),
            (2,5),
            (3,4), (3,6),
            (4,5), (4,7),
            (5,8),
            (6,7),
            (7,8)
           ]

In [2]:
# utilize the recursive path function from documentation with minor tweak
# Breakdown: 
# If start == end, then we are finished & path is found
# If start is not in graph, then we can't go anywhere (this would mean the arcs point to start instread of away)
# Otherwise, iterate over node options recursively -> working backwards to determine if we can navigate form s -> e
def find_path(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return path
        if start not in graph:
            return None
        for node in graph[start]:
            if node not in path:
                newpath = find_path(graph, node, end, path)
                if newpath: 
                    return newpath
        return None

def randomRoute(l_tup):
    """Randomly flip edges to generate list of of arcs"""
    out_tup = []
    for edge in l_tup:
        if bi(1, 0.5, 1):
            edge = (edge[1], edge[0])
        out_tup.append(edge)
    return out_tup


def graphBuild(l_tup):
    routes = defaultdict( list )

    for k,v in l_tup:
        routes[k].append(v)
    
    return routes

### Confirm Path Traversal Works:

Trying a few examples:
1) Original path -> we know this works

2) Impossible patth -> No way to move out of vertex 1

3) Path with infinite loop -> ensure we break out of infinite recursion

In [3]:
start = 0
end = 8

# original list of edges all east / south
l_tuples = [(0,1), (0,3),
            (1,2), (1,4),
            (2,5),
            (3,4), (3,6),
            (4,5), (4,7),
            (5,8),
            (6,7),
            (7,8)
           ]

# build graph
graph = graphBuild(l_tuples)

print(find_path(graph, start, end))
assert(find_path(graph, start, end) != None)

# check an impossible example now 
# 0 - 1 exists, but then 1 is flipped to go backwards
# 0 - 3 no longer exists (only goes up)

l_tuples = [(0,1), (3,0),
            (2,1), (4,1),
            (2,5),
            (3,4), (3,6),
            (4,5), (4,7),
            (5,8),
            (6,7),
            (7,8)
           ]

# build graph
graph = graphBuild(l_tuples)

print(find_path(graph, start, end))
assert(find_path(graph, start, end) == None)

# check an infinite loop example now 
# 0 - 1 , 1 -4 , 4 - 3, 3 - 0

# each tuple represents a graph arc
l_tuples = [(0,1), (3,0),
            (2,1), (1,4),
            (2,5),
            (4, 3), (6,3),
            (5,4), (7,4),
            (5,8),
            (6,7),
            (7,8)
           ]

# build graph
graph = graphBuild(l_tuples)

print(find_path(graph, start, end))
assert(find_path(graph, start, end) == None)

[0, 1, 2, 5, 8]
None
None


#### Sim Progression: 

- see how the prob of success shifts over time 

In [4]:
# original list of edges all east / south
l_tuples = [(0,1), (0,3),
            (1,2), (1,4),
            (2,5),
            (3,4), (3,6),
            (4,5), (4,7),
            (5,8),
            (6,7),
            (7,8)
           ]

sim_list = [100,1_000,10_000, 100_000, 250_000, 500_000, 1_000_000, 2_000_000]

for sims in sim_list:

    success = 0

    for _ in range(sims):

        # randomize route
        out = randomRoute(l_tuples)

        # convert to graph 
        routes = graphBuild(out)

        # graph traversal
        try:
            all_paths = find_path(routes, 0,8)
            if all_paths != None:
                #print(all_paths)
                success += 1
        except:
            print("Error")


    print(f"Probability of success for sim size {sims}: {success/sims:.4f}")

Probability of success for sim size 100: 0.3800
Probability of success for sim size 1000: 0.2760
Probability of success for sim size 10000: 0.2749
Probability of success for sim size 100000: 0.2771
Probability of success for sim size 250000: 0.2770
Probability of success for sim size 500000: 0.2767
Probability of success for sim size 1000000: 0.2770
Probability of success for sim size 2000000: 0.2765


### Analytical Solution:

- Determine all possible combinations of routes (think of them as + vs -)
- Determine total routes that allow for Start -> Finish 

#### Arcs:
- 12 total arcs which can be (+) or (-) ---> 2^12 combinations (I think) 
- Potential routes need to include 2 horizontal (+) and 2 vertical (+)....

In [7]:
2**12

4096